In [20]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# File path of the input data
input_path = r"C:\Users\bedro\OneDrive - Syracuse University\Desktop\PhD courses\CEE 609 Env Data Science\project\Linear regression model\Filtered_Extraction_Accuracy.xlsx"

# Load the data
df = pd.read_excel(input_path)

# Filter rows where 'File Format' is 'tif'
df_filtered = df[df['File Format'] == 'tif'].copy()

# Preserve original values before scaling
df_filtered.loc[:, 'Original Resolution (DPI)'] = df_filtered['Resolution (DPI)']
df_filtered.loc[:, 'Original Noise Level (%)'] = df_filtered['Noise Level (%)']

# Convert 'Contract Date' to numerical format (days since a reference date)
df_filtered.loc[:, 'Contract Date (Numeric)'] = (
    pd.to_datetime(df_filtered['Contract Date']) - pd.to_datetime('2000-01-01')
).dt.days

# Scale the predictors
scaler = MinMaxScaler()
df_filtered[['Resolution (DPI)', 'Noise Level (%)', 'Contract Date (Numeric)']] = scaler.fit_transform(
    df_filtered[['Resolution (DPI)', 'Noise Level (%)', 'Contract Date (Numeric)']]
)

# Save the filtered and scaled data
output_path = r"C:\Users\bedro\OneDrive - Syracuse University\Desktop\Filtered_Scaled_Data_with_Original_Values.xlsx"
df_filtered.to_excel(output_path, index=False)
print(f"Filtered and scaled data saved to: {output_path}")


Filtered and scaled data saved to: C:\Users\bedro\OneDrive - Syracuse University\Desktop\Filtered_Scaled_Data_with_Original_Values.xlsx


In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# File path of the filtered and scaled data
data_path = r"C:\Users\bedro\OneDrive - Syracuse University\Desktop\Filtered_Scaled_Data_with_Original_Values.xlsx"

# Load the data
df = pd.read_excel(data_path)

# Define predictors (X) and target (y)
X = df[['Resolution (DPI)', 'Noise Level (%)', 'Contract Date (Numeric)']]
y = df['Extraction Accuracy']  # Assuming 1 for correct and 0 for incorrect

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the logistic regression model
model = LogisticRegression()

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Display model coefficients
print("\nModel Coefficients:")
for feature, coef in zip(X.columns, model.coef_[0]):
    print(f"{feature}: {coef:.4f}")


Model Accuracy: 0.70

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.01      0.02        87
           1       0.70      1.00      0.82       199

    accuracy                           0.70       286
   macro avg       0.85      0.51      0.42       286
weighted avg       0.79      0.70      0.58       286


Confusion Matrix:
[[  1  86]
 [  0 199]]

Model Coefficients:
Resolution (DPI): 0.3784
Noise Level (%): -1.6187
Contract Date (Numeric): 2.4071
